In [1]:
import cv2
import numpy as np
import os 

# Load Yolo
print("LOADING YOLO")
net = cv2.dnn.readNet("yolov4-custom_last.weights", "yolov4-custom.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
#save all the names in file o the list classes
classes = []
with open("obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
#get layers of the network
layer_names = net.getLayerNames()
#Determine the output layer names from the YOLO model 
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]
print("YOLO LOADED")
GREEN = (0,255,0) 
RED = (0,0,255)
BLUE = (255,0,0)
BLACK = (0,0,0)
YELLOW =(0,255,255)
WHITE = (255,255,255)

LOADING YOLO
YOLO LOADED


In [2]:
classes

['trafficlight', 'speedlimit', 'crosswalk', 'stop']

In [3]:
video_capture = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    re,img = video_capture.read()
    #img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    # USing blob function of opencv to preprocess image
    blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416),
     swapRB=True, crop=False)
    #Detecting objects
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    #We use NMS function in opencv to perform Non-maximum Suppression
    #we give it score threshold and nms threshold as arguments.
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    #font = cv2.FONT_HERSHEY_DUPLEX
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            if label=="trafficlight":
                cv2.rectangle(img, (x, y), (x + w, y + h), (0,255,0), 1)
                cv2.putText(img, label, (x, y -20), 0, 1/2,(255,0,0), 1)
            elif label=="speedlimit":
                cv2.rectangle(img, (x, y), (x + w, y + h), (0,0,255), 1)
                cv2.putText(img, label, (x, y -20), 0, 1/2,(255,0,0), 1)
            elif label=="crosswalk":
                cv2.rectangle(img, (x, y), (x + w, y + h), (125,125,125), 1)
                cv2.putText(img, label, (x, y -20), 0, 1/2,(255,0,0), 1)
            elif label=="stop":
                cv2.rectangle(img, (x, y), (x + w, y + h), (125,125,125), 1)
                cv2.putText(img, label, (x, y -20), 0, 1/2,(255,0,0), 1)
                
            #cv2.putText(img,'Distance is '+str(distance)+' Inch',(x, y + 15), font, 2, YELLOW, 2)
#             else:
#                 cv2.rectangle(img, (x, y), (x + w, y + h), GREEN, 2)
#                 cv2.putText(img, "without_mask", (x, y -20), font, 1/2, BLUE, 1)
#                 #cv2.putText(img,'Distance is '+str(distance)+' Inch',(x, y + 15), font, 2, YELLOW, 2)
#                 cv2.putText(img,str(distance)+' Inch',(x, y - 5), font, 1/2, BLUE, 1)       
                
                

    cv2.imshow("Image",img)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()